# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from IPython.display import display, HTML

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    path_root_star = os.path.join(root,'*')
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(path_root_star)
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## Create the table using an appropriate primary key.
## As the filter will be on sessionId and itemInSession our primary key will be made of those attributes
query = 'CREATE TABLE IF NOT EXISTS session_item'
query += '(sessionId INT, itemInSession INT, artist TEXT, song TEXT, duration FLOAT, PRIMARY KEY(sessionId, itemInSession))'
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_item (sessionId, itemInSession, artist, song, duration)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        # Line fields by index:
        # 0 artist 
        # 1 firstName of user
        # 2 gender of user
        # 3 item number in session
        # 4 last name of user
        # 5 length of the song
        # 6 level (paid or free song)
        # 7 location of the user
        # 8 sessionId
        # 9 song title
        # 10 userId

        ## Print out match for query for later comparison
        if line[8] == '338' and line[3] == '4': 
            print(line[0], line[9], line[5], sep=",")

        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

Faithless,Music Matters (Mark Knight Dub),495.3073


#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Add in the SELECT statement to verify the data was entered into the table
query = 'SELECT artist, song, duration FROM session_item WHERE sessionId = 338 AND itemInSession = 4'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame([[row.artist, row.song, row.duration] for row in rows], columns=["Artist", "Song", "Duration"])
display(HTML(df.to_html(index=False)))

Artist,Song,Duration
Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## Create the table using an appropriate primary key and clustering column.
## As the filter will be on userid and sessionId our primary key will be made of those attributes.
## As the results should be sorted by the itemInSession include that attribute in clustering column.
query = 'CREATE TABLE IF NOT EXISTS user_session'
query += '(userid INT, sessionid INT, itemInSession INT, artist TEXT, song TEXT, firstname TEXT, lastname TEXT, PRIMARY KEY((userid, sessionId), itemInSession))'
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session (userid, sessionid, itemInSession, artist, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

        # Line fields by index:
        # 0 artist 
        # 1 firstName of user
        # 2 gender of user
        # 3 item number in session
        # 4 last name of user
        # 5 length of the song
        # 6 level (paid or free song)
        # 7 location of the user
        # 8 sessionId
        # 9 song title
        # 10 userId

        ## Print out match for query for later comparison
        if line[10] == '10' and line[8] == '182': 
            print(line[0], line[9], line[3], line[1], line[4], sep=",")
            
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

Down To The Bone,Keep On Keepin' On,0,Sylvie,Cruz
Three Drives,Greece 2000,1,Sylvie,Cruz
Sebastien Tellier,Kilometer,2,Sylvie,Cruz
Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio Edit),3,Sylvie,Cruz


In [13]:
query = 'SELECT artist, song, itemInSession, firstname, lastname FROM user_session WHERE userid = 10 AND sessionid = 182'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame([[row.artist, row.song, row.iteminsession, row.firstname, row.lastname] for row in rows], columns=["Artist", "Song", "Item in Session", "First Name", "Last Name"])
display(HTML(df.to_html(index=False)))

Artist,Song,Item in Session,First Name,Last Name
Down To The Bone,Keep On Keepin' On,0,Sylvie,Cruz
Three Drives,Greece 2000,1,Sylvie,Cruz
Sebastien Tellier,Kilometer,2,Sylvie,Cruz
Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,3,Sylvie,Cruz


In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Create the table using an appropriate primary key.
## As the filter will be on song and user our primary key will be made of those attributes.  Here userid is a proxy for first
## and last names together.
query = 'CREATE TABLE IF NOT EXISTS song_users'
query += '(song TEXT, userid INT, firstname TEXT, lastname TEXT, PRIMARY KEY(song, userid))'
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_users (song, userid, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        # Line fields by index:
        # 0 artist 
        # 1 firstName of user
        # 2 gender of user
        # 3 item number in session
        # 4 last name of user
        # 5 length of the song
        # 6 level (paid or free song)
        # 7 location of the user
        # 8 sessionId
        # 9 song title
        # 10 userId

        ## Print out match for query for later comparison
        if line[9] == 'All Hands Against His Own': 
            print(line[1], line[4])

        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [16]:
query = "SELECT firstname, lastname FROM song_users WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame([[row.firstname, row.lastname] for row in rows], columns=["First Name", "Last Name"])
display(HTML(df.to_html(index=False)))

First Name,Last Name
Jacqueline,Lynch
Tegan,Levine
Sara,Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions

In [18]:
query = 'DROP TABLE song_users'
try:
    session.execute(query)
except Exception as e:
    print(e)

query = 'DROP TABLE user_session'
try:
    session.execute(query)
except Exception as e:
    print(e)

query = 'DROP TABLE session_item'
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()